In [43]:
import pandas as pd
import psycopg2

In [2]:
df_transaction = pd.read_excel('data/customer_and_transaction__2_.xlsx', sheet_name='transaction')
df_transaction.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92


In [3]:
df_customer = pd.read_excel('data/customer_and_transaction__2_.xlsx', sheet_name='customer')
df_customer.head(2)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10


In [4]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      19640 non-null  object        
 5   order_status      20000 non-null  object        
 6   brand             19803 non-null  object        
 7   product_line      19803 non-null  object        
 8   product_class     19803 non-null  object        
 9   product_size      19803 non-null  object        
 10  list_price        20000 non-null  float64       
 11  standard_cost     19803 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.8+ MB


In [5]:
df_transaction.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost'],
      dtype='object')

In [9]:
# Cоздаем таблицу product
df_product = df_transaction[['product_id', 'brand', 
                             'product_line','product_class', 
                             'product_size', 'list_price', 'standard_cost']].drop_duplicates().sort_values(by=['product_id'], 
                                                                            ignore_index=True)

# Добавляем суррогатный ключ new_product_id, так как product_id не уникальный
df_product = df_product.assign(row_number=range(1, len(df_product)+1))

df_product.rename(columns={'row_number': 'new_product_id'}, inplace=True)
print('Получили новую таблицу product с уникальным ключом - new_product_id')
df_product.head()


Получили новую таблицу product с уникальным ключом - new_product_id


,product_id,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_id
0,0,NaN,NaN,NaN,NaN,867.92,NaN,1
1,0,NaN,NaN,NaN,NaN,1697.27,NaN,2
2,0,NaN,NaN,NaN,NaN,1300.96,NaN,3
3,0,OHM Cycles,Standard,high,medium,227.88,136.73,4
4,0,NaN,NaN,NaN,NaN,126.36,NaN,5


In [14]:
print('Всего получили 399 уникальных ключей:')
df_product.info()

Всего получили 399 уникальных ключей:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      399 non-null    int64  
 1   brand           203 non-null    object 
 2   product_line    203 non-null    object 
 3   product_class   203 non-null    object 
 4   product_size    203 non-null    object 
 5   list_price      399 non-null    float64
 6   standard_cost   203 non-null    float64
 7   new_product_id  399 non-null    int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 25.1+ KB


In [18]:
# Добавляем new_product_id в таблицу transaction
df_transaction_new = df_transaction.merge(df_product, 
                                          left_on=['product_id', 'brand', 'product_line','product_class', 
                                                   'product_size', 'list_price', 'standard_cost'], 
                                          right_on=['product_id', 'brand', 'product_line','product_class', 
                                                   'product_size', 'list_price', 'standard_cost'], 
                                          how='left')

# Удаляем лишние данные
df_transaction_new.drop(columns=['brand', 'product_line','product_class', 'product_id',
                                 'product_size', 'list_price', 'standard_cost'], inplace=True)

df_transaction_new.head()

,transaction_id,customer_id,transaction_date,online_order,order_status,new_product_id
0,1,2950,2017-02-25,False,Approved,226
1,2,3120,2017-05-21,True,Approved,228
2,3,402,2017-10-16,False,Approved,290
3,4,3135,2017-08-31,False,Approved,375
4,5,787,2017-10-01,True,Approved,357


*** Customer ***

In [22]:
#
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

In [31]:
# Cоздаем таблицу job
df_job = df_customer[['job_title', 'job_industry_category']].drop_duplicates().sort_values(by=['job_title'], 
                                                                                           ignore_index=True)

# Добавляем суррогатный ключ new_product_id, так как product_id не уникальный
df_job = df_job.assign(row_number=range(1, len(df_job)+1))

df_job.rename(columns={'row_number': 'job_id'}, inplace=True)
print('Получили новую таблицу job с уникальным ключом - job_id')
df_job.head(3)

Получили новую таблицу job с уникальным ключом - job_id


,job_title,job_industry_category,job_id
0,Account Coordinator,Argiculture,1
1,Account Coordinator,Financial Services,2
2,Account Coordinator,Health,3


In [32]:
# Cоздаем таблицу address
df_address = df_customer[['address', 'postcode', 'state', 'country']].drop_duplicates().sort_values(by=['address'], 
                                                                                           ignore_index=True)

# Добавляем суррогатный ключ new_product_id, так как product_id не уникальный
df_address = df_address.assign(row_number=range(1, len(df_address)+1))

df_address.rename(columns={'row_number': 'address_id'}, inplace=True)
print('Получили новую таблицу address с уникальным ключом - address_id')
df_address.head(3)

Получили новую таблицу address с уникальным ключом - address_id


,address,postcode,state,country,address_id
0,0 3rd Road,4060,QLD,Australia,1
1,0 American Ash Parkway,3175,VIC,Australia,2
2,0 Arapahoe Court,4305,QLD,Australia,3


In [44]:
# Добавляем new_product_id в таблицу transaction
df_customer_new = df_customer.merge(df_job, 
                                    left_on=['job_title', 'job_industry_category'], 
                                    right_on=['job_title', 'job_industry_category'], 
                                    how='left')

df_customer_new = df_customer_new.merge(df_address, 
                                    left_on=['address', 'postcode', 'state', 'country'], 
                                    right_on=['address', 'postcode', 'state', 'country'], 
                                    how='left')
# Удаляем лишние данные
df_customer_new.drop(columns=['address', 'postcode', 'state', 'country', 
                              'job_title', 'job_industry_category'], inplace=True)

df_customer_new.head(3)

,customer_id,first_name,last_name,gender,DOB,wealth_segment,deceased_indicator,owns_car,property_valuation,job_id,address_id
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Mass Customer,N,Yes,10,364,293
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Mass Customer,N,Yes,10,102,2443
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Mass Customer,N,Yes,9,700,32


In [36]:
df_customer_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         4000 non-null   int64 
 1   first_name          4000 non-null   object
 2   last_name           3875 non-null   object
 3   gender              4000 non-null   object
 4   DOB                 3913 non-null   object
 5   wealth_segment      4000 non-null   object
 6   deceased_indicator  4000 non-null   object
 7   owns_car            4000 non-null   object
 8   property_valuation  4000 non-null   int64 
 9   job_id              4000 non-null   int64 
 10  address_id          4000 non-null   int64 
dtypes: int64(4), object(7)
memory usage: 343.9+ KB


In [95]:
# Устанавливаем соединение с базой данных
conn = psycopg2.connect(user="postgres", 
                        password="1234554321live##Test", 
                        host="localhost", 
                        database="postgres")

In [76]:
# Создаем объект курсора
cur = conn.cursor()

In [70]:
# Переводим даты в тип datetime для таблицы Customers и заменяем пустое значение на '1900-01-01'
df_customer_new['DOB'] = pd.to_datetime(df_customer_new['DOB'])
df_customer_new['DOB'] = df_customer_new['DOB'].fillna(pd.Timestamp('1900-01-01'))  # Заменяем отсутствующие значения на заданную дату

In [71]:
# Внесение данных в таблицу Customers
for index,row in df_customer_new.iterrows():
      cur.execute('''insert into public.customers (customer_id, first_name, last_name,
                  gender, DOB, wealth_segment, owns_car, property_valuation, job_id, address_id) 
                  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''', 
                  (row['customer_id'], row['first_name'], row['last_name'], 
                   row['gender'], row['DOB'], row['wealth_segment'], 
                   row['owns_car'], row['property_valuation'], row['job_id'], 
                   row['address_id']))

# Закрываем курсор и подтверждаем изменения в базе данных
cur.close()
conn.commit()

In [83]:
# Создаем объект курсора
cur = conn.cursor()
# Внесение данных в таблицу Jobs
for index,row in df_job.iterrows():
      cur.execute('''insert into public.jobs (job_id, job_title, job_industry_category) 
                  VALUES (%s,%s,%s)''', 
                  (row['job_id'], row['job_title'], row['job_industry_category']))

# Закрываем курсор и подтверждаем изменения в базе данных
cur.close()
conn.commit()

In [85]:
# Создаем объект курсора
cur = conn.cursor()
# Внесение данных в таблицу Addresses
for index,row in df_address.iterrows():
      cur.execute('''insert into public.addresses (address_id, address,	postcode,	state, country) 
                  VALUES (%s,%s,%s,%s,%s)''', 
                  (row['address_id'], row['address'], row['postcode'], row['state'], row['country']))

# Закрываем курсор и подтверждаем изменения в базе данных
cur.close()
conn.commit()

In [91]:
# Переводим даты в тип datetime
df_transaction_new['transaction_date'] = pd.to_datetime(df_transaction_new['transaction_date'])
# Создаем объект курсора
cur = conn.cursor()
# Внесение данных в таблицу Transactions
for index,row in df_transaction_new.iterrows():
      cur.execute('''insert into public.transactions (transaction_id,	customer_id,	transaction_date,	
                  online_order,	order_status,	new_product_id) 
                  VALUES (%s,%s,%s,%s,%s,%s)''', 
                  (row['transaction_id'], row['customer_id'], row['transaction_date'], 
                   row['online_order'], row['order_status'], row['new_product_id']))

# Закрываем курсор и подтверждаем изменения в базе данных
cur.close()
conn.commit()

In [96]:
# Создаем объект курсора
cur = conn.cursor()
# Внесение данных в таблицу Products
for index,row in df_product.iterrows():
      cur.execute('''insert into public.products (new_product_id, product_id,	brand,	product_line,	
                  product_class,	product_size,	list_price,	standard_cost) 
                  VALUES (%s,%s,%s,%s,%s,%s,%s,%s)''', 
                  (row['new_product_id'], row['product_id'], row['brand'], 
                   row['product_line'], row['product_class'], row['product_size'],
                   row['list_price'], row['standard_cost']))

# Закрываем курсор и подтверждаем изменения в базе данных
cur.close()
conn.commit()